### Webscrapping from 4f.com

Let download prices and names all new produckts from 4F.com website

In [98]:
from lxml import html
import requests 
import pandas as pd

In [220]:
# sample link 
url = 'https://4f.com.pl/nowosci/strona-1.html?product_list_limit=120'

In [221]:
# get whole structure of data
page = requests.get(url) 

In [222]:
# show not parsed text
print(page.content)

b' <!doctype html><html lang="pl"><head ><script> var BASE_URL = \'https\\u003A\\u002F\\u002F4f.com.pl\\u002F\'; var require = { \'baseUrl\': \'https\\u003A\\u002F\\u002F4f.com.pl\\u002Fstatic\\u002Fversion1638346793\\u002Ffrontend\\u002FFwc\\u002Fotcf\\u002Fpl_PL\' };</script> <meta charset="utf-8"/>\n<meta name="title" content="Nowo\xc5\x9bci - Nowo\xc5\x9bci - 4F Odzie\xc5\xbc sportowa"/>\n<meta name="description" content="Nowo\xc5\x9bci | Nowa kolekcja odzie\xc5\xbcy sportowej w sklepie internetowym 4F | \xe2\x9c\x85 5% zni\xc5\xbcki dla zarejestrowanych \xe2\x9c\x85 Darmowa dostawa od 99 z\xc5\x82"/>\n<meta name="robots" content="INDEX,FOLLOW"/>\n<meta name="viewport" content="width=device-width, initial-scale=1"/>\n<meta name="format-detection" content="telephone=no"/>\n<title>Nowo\xc5\x9bci - Nowo\xc5\x9bci - 4F Odzie\xc5\xbc sportowa</title>\n<link  rel="stylesheet" type="text/css"  media="all" href="https://4f.com.pl/static/version1638346793/_cache/merged/c9a5bc90f3da258f720c0

In [224]:
# let parse to html object
tree = html.fromstring(page.content)
print(tree)

<Element html at 0x2e76a090>


In [229]:
# get the names
xpath_selector = '//*[@id="maincontent"]/div[2]/div[1]/div[6]/ol/li/div/strong/a/text()'
products = tree.xpath(xpath_selector)
products[:4]

['Bluza oversize ze sztruksu damska',
 'Bluza dresowa z otworem na plecach damska',
 'Gładki golf z rękawem 1/2 damski',
 'Polar z kapturem regular męski']

In [230]:
# get the prices
xpath_selector_prices = '/html/body/div[3]/main/div[2]/div[1]/div[6]/ol/li/div/div[3]/div/span/span/span/text()'
prices = tree.xpath(xpath_selector_prices)
prices[:4]




['129,99\xa0PLN', '149,99\xa0PLN', '69,99\xa0PLN', '229,99\xa0PLN']

In [231]:
# let automate getting data from all website, containing "Nowosci" from website

# empty dataframe to collect data
datatable= pd.DataFrame({"Names" : [],
             "Prices" :[]})


for i in range(1,1000):  # random big number to keep iterate until we reach limit
    #our website with all possible subsites 
    url =  "https://4f.com.pl/nowosci/strona-" + str(i) + ".html?product_list_limit=120"
    
    # get whole structure of data
    page = requests.get(url)

    # let parse it to html object
    tree = html.fromstring(page.content)
    
    # get the names from website
    xpath_selector_names = '//*[@id="maincontent"]/div[2]/div[1]/div[6]/ol/li/div/strong/a/text()'
    products = tree.xpath(xpath_selector_names)
    
    # get the prices from website
    xpath_selector_prices = '/html/body/div[3]/main/div[2]/div[1]/div[6]/ol/li/div/div[3]/div/span/span/span/text()'
    prices = tree.xpath(xpath_selector_prices)
    
    # join names and prices together
    for i in range(0, len(products)):
        datatable = datatable.append({"Names" : products[i],
                    "Prices" : prices[i]},
                    ignore_index = True)
    
    # check if it is last page
    xpath_selector_next_page = '/html/body/div[3]/main/div[2]/div[1]/div[7]/div[4]/ul/li/a/span[2]/text()'
    next_page = tree.xpath(xpath_selector_next_page)
    
    if next_page[-1] != 'Następne':
        break 

In [233]:
# print the result
datatable

,Names,Prices
0,Bluza oversize ze sztruksu damska,"129,99 PLN"
1,Bluza dresowa z otworem na plecach damska,"149,99 PLN"
2,Gładki golf z rękawem 1/2 damski,"69,99 PLN"
3,Polar z kapturem regular męski,"229,99 PLN"
4,Polar bez kaptura sherpa męski,"249,99 PLN"
...,...,...
881,Skarpetki dziewczęce (3 pary),"39,99 PLN"
882,Skarpetki damskie (3 pary),"39,99 PLN"
883,Skarpetki damskie (2 pary),"19,99 PLN"
884,Skarpetki damskie (2 pary),"19,99 PLN"
